In [1]:
import requests
from bs4 import BeautifulSoup
import re
from json import dump
from collections import defaultdict
import time
import random
import pickle
import json

postcode_url = 'https://www.worldpostalcodes.org/l1/en/au/australia/list/r1/list-of-postcodes-in-victoria'

response = requests.request("GET", postcode_url, headers=None, data=None)
bs_object = BeautifulSoup(response.text, 'html.parser')
table_tag = bs_object.find('table')
tr_tags = table_tag.find_all('tr')[2:]

all_postcode = []
for tr_tag in tr_tags:
    postcode = tr_tag.next.next.text
    if postcode != '':
        all_postcode.append(int(postcode))

In [2]:
postcode = 3000
page = 1
url = f"https://www.domain.com.au/rent/?postcode={postcode}&page={page}"

payload={}
headers = {
    'user-agent': 'ozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, timeout=5)


In [ ]:
urls = []

## loop over all postcodes in Vic
for postcode in all_postcode:
    print(f'postcode: {postcode}')
    page = 1

    ## iterate all available pages
    while True:
        url = f"https://www.domain.com.au/rent/?postcode={postcode}&page={page}"

        payload={}
        headers = {
        'user-agent': 'ozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
        }

        response = requests.request("GET", url, headers=headers, data=payload, timeout=5)

        ## parse the web content
        bs_object = BeautifulSoup(response.text, 'html.parser')

        target_ul = bs_object.find('ul', attrs={'data-testid': "results"})
        # if page is not found, terminate current loop 
        if target_ul is None:
            print('page not found')
            break

        ## get property detail links
        all_links = target_ul.find_all('a')
        for links in all_links:
            urls.append(links['href'])
        
        page += 1
        if page % 10 == 0:
            print(page)

        # time.sleep(random.uniform(0, 2))
## Save urls to disk

with open('urls.pickle', 'wb') as f:
    pickle.dump(urls, f)

In [349]:
properties_dict = {}
failed_urls = []

## loop over all urls
for i, url in enumerate(urls):
    if i % 100 == 0:
        print(f"Urls NO. {i}")
    
    ## get the property content
    payload={}
    headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    response = requests.request("GET", url, headers=headers, data=payload, timeout=10)
    
    bs_object = BeautifulSoup(response.text, 'html.parser')
    ## get price info
    price_tag = bs_object.find('div', attrs={'data-testid': "listing-details__summary-title"})
    if price_tag is None:
        failed_urls.append(url)
        continue
    else:
        price_text = bs_object.find('div', attrs={'data-testid': "listing-details__summary-title"}).text
    ## get address info
    address_text = bs_object.find('h1', attrs={'class': 'css-164r41r'}).text
    ## get beds, baths and parking info
    property_features_tag = bs_object.find('div', attrs={'data-testid': "property-features-wrapper"})
    try:
        if len(list(property_features_tag.children)) > 0:
            num_bedrooms_text = property_features_tag.next.next.text
            num_bathrooms_text = property_features_tag.next.next_sibling.next.text
            num_parking_text = property_features_tag.next.next_sibling.next_sibling.next.text
        else:
            num_bedrooms_text = ""
            num_bathrooms_text = ""
            num_parking_text = ""
    except:
        continue
    ## get propery type
    property_type_text = bs_object.find('div', attrs={'data-testid': "listing-summary-property-type"}).next.text
    ## get propery features
    property_tag = bs_object.find('ul', attrs={'class': 'css-4ewd2m'})
    if property_tag is not None:
        property_features = [child.text for child in property_tag.children]
    else:
        property_features = []

    ## save to a dict
    property_dict = dict()
    property_dict['price'] = price_text
    property_dict['address'] = address_text
    property_dict['postcode'] = address_text[-4:]
    property_dict['bedrooms'] = num_bedrooms_text
    property_dict['baths'] = num_bathrooms_text
    property_dict['parking'] = num_parking_text
    property_dict['property_type'] = property_type_text
    property_dict['property_features'] = property_features

    properties_dict[url] = property_dict

    # time.sleep(random.uniform(0, 0.3))

## output to disk
with open('properties_dict.json', 'w') as f:
    json.dump(properties_dict, f, indent=2)

Urls NO. 5400
Urls NO. 5500
Urls NO. 5600
Urls NO. 5700
Urls NO. 5800
Urls NO. 5900
Urls NO. 6000
Urls NO. 6100
Urls NO. 6200
Urls NO. 6300
Urls NO. 6400
Urls NO. 6500
Urls NO. 6600
Urls NO. 6700
Urls NO. 6800
Urls NO. 6900
Urls NO. 7000
Urls NO. 7100
Urls NO. 7200
Urls NO. 7300
Urls NO. 7400
Urls NO. 7500
Urls NO. 7600
Urls NO. 7700
Urls NO. 7800
Urls NO. 7900
Urls NO. 8000
Urls NO. 8100
Urls NO. 8200
Urls NO. 8300
Urls NO. 8400
Urls NO. 8500
Urls NO. 8600
Urls NO. 8700
Urls NO. 8800
Urls NO. 8900
Urls NO. 9000
Urls NO. 9100
Urls NO. 9200
Urls NO. 9300
Urls NO. 9400
Urls NO. 9500
Urls NO. 9600
Urls NO. 9700
Urls NO. 9800
Urls NO. 9900
Urls NO. 10000
Urls NO. 10100
Urls NO. 10200
Urls NO. 10300
Urls NO. 10400
Urls NO. 10500
Urls NO. 10600
Urls NO. 10700
Urls NO. 10800
Urls NO. 10900
Urls NO. 11000
Urls NO. 11100
Urls NO. 11200
Urls NO. 11300
Urls NO. 11400
Urls NO. 11500
Urls NO. 11600
Urls NO. 11700
Urls NO. 11800
Urls NO. 11900
Urls NO. 12000
Urls NO. 12100
Urls NO. 12200
Urls NO. 12

In [404]:
with open('properties_dict.json', 'r') as f:
    properties_dict_copy = json.load(f)

## postprocess raw data
for url, item in properties_dict_copy.items():
    ## get price
    price = re.findall(r'\$(\d*\.?\d{0,2})?', re.split('[/ ]', item['price'])[0])
    if len(price) != 0 and price[0] != '':
        price[0] = price[0].replace(',', '')
        if price[0].endswith('pw') or price[0].endswith('/w'):     
            item['price'] = float(price[0][:-2])
        else:
            item['price'] = float(price[0])
    else:
        item['price'] = ''
    ## get bedrooms
    bedrooms = re.findall(r'(\d{1,2})', item['bedrooms'])
    if len(bedrooms) != 0:
        item['bedrooms'] = int(bedrooms[0])
    else:
        item['bedrooms'] = 0
    ## get baths 
    baths = re.findall(r'(\d{1,2})', item['baths'])
    if len(baths) != 0:
        item['baths'] = int(baths[0])
    else:
        item['baths'] = 0
    ## get parking  
    parking = re.findall(r'(\d{1,2})', item['parking'])
    if len(parking) != 0:
        item['parking'] = int(parking[0])
    else:
        item['parking'] = 0

## output to disk
with open('properties_dict_processed.json', 'w') as f:
    json.dump(properties_dict_copy, f, indent=2)